# 로지스틱 회귀

In [1]:
import pandas as pd

In [3]:
fish = pd.read_csv('https://bit.ly/fish_csv')

In [4]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [5]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


* 몇 개의 종류의 Species의 특성이 있는지 보여준다. 

In [7]:
fish_input = fish[['Weight', 'Length','Diagonal','Height','Width']].to_numpy()

In [8]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [9]:
fish_target = fish['Species'].to_numpy()

In [11]:
#5개 특성에 따른 결과값(Species)를 예측해보자.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state = 42)

In [15]:
#표준화
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [16]:
# 전에 배웠던 k-최근접 이웃 알고리즘으로 분류해보기
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


* 과소적합의 경향이 보인다. 

In [19]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


* 타깃값을 사이킷런 모델에 전달하게 되면 순서가 자동으로 알파벳 순서로 매겨집니다. 

In [20]:
# test_scaled의 5개의 샘플 값 예측
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [21]:
# predict_proba() : 클래스별 확률 값을 반환한다. 
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=4))

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


* 출력 순서는 kn_classes_ 속성과 같다. 

## 로지스틱 회귀로 이진 분류 수행하기 

In [22]:
# 불리언 인덱싱
char_arr = np.array(['A','B','C','D','E'])
print(char_arr[[True,False,True,False,False]])

['A' 'C']


* True인 값들만 출력된다. 

In [25]:
# 불리언 인덱싱으로 도미와 빙어 행 골라내기
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smlet = train_scaled[bream_smelt_indexes]
target_bream_smlet = train_target[bream_smelt_indexes]

In [26]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smlet,target_bream_smlet)

LogisticRegression()

In [29]:
print(lr.predict(train_bream_smlet[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [30]:
print(lr.predict_proba(train_bream_smlet[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


* 이때 첫 번째 열이 음성클래스(0)에 대한 확률, 두 번째 열이 양성클래스(1)에 대한 확률

In [31]:
# 로지스틱 회귀 모델이 학습한 방정식의 계수
print(lr.coef_,lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


* z = -0.404 * (Weight) - 0.576 * (length) - 0.663 * (Diagonal) - 1.013 * (Height) - 0.732 * (Width) - 2.161

In [34]:
# z 값 출력
decisions = lr.decision_function(train_bream_smlet[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


In [36]:
# expit()함수를 사용하여 시그모이드 함수 값을 구해보자(0~1)  음성클래스( < = 0.5 ) , 양성클래스 (  > 0.5 )
from scipy.special import expit
print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


## 로지스틱 회귀로 다중 분류 수행하기 

In [37]:
lr = LogisticRegression(C = 20, max_iter = 1000) 
lr.fit(train_scaled,train_target)
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))

0.9327731092436975
0.925


* 로지스터 회귀는 반복 알고리즘을 사용한다. max_iter는 반복 횟수를 지정하고 기본값을 100입니다. 
*  C = 규제 정도, 작을수록 규제가 커진다. 

In [38]:
# 5개의 샘플 예측 결과
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [39]:
# 5개의 샘플에 대한 예측 확률
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [40]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [43]:
print(lr.coef_)
print(lr.intercept_)

[[-1.49002298 -1.02913987  2.59346473  7.70357182 -1.20069792]
 [ 0.19618357 -2.01066946 -3.7797786   6.50491214 -1.99483384]
 [ 3.56279548  6.34357901 -8.48971543 -5.75757183  3.79307646]
 [-0.10458074  3.60318964  3.93067746 -3.61735617 -1.75069776]
 [-1.4006114  -6.07503751  5.25969076 -0.8722058   1.86043601]
 [-1.38526445  1.49215091  1.3922659  -5.67733837 -4.40097226]
 [ 0.62150052 -2.32407272 -0.90660482  1.7159882   3.69368931]]
[-0.09205143 -0.26290902  3.25101331 -0.14743174  2.65497973 -6.78782302
  1.38422217]


In [44]:
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


* 5개의 특성 = 5 열
* z의 값 7개를 계산 = 7행
* 이진 분류의 경우 확률 계산을 시그모이드 함수로 하였다.
* but 다중 분류의 경우 확률 계산을 소프트맥스 함수로 한다. 

In [46]:
# z값 계산
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals = 2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [47]:
from scipy.special import softmax
proba = softmax(decision, axis = 1)
print(np.round(proba, decimals = 3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


* axis = 1 (행에 대하여 소프트 맥스를 계산한다. - z들에 대해 계산해야 하므로) 
* axis 지정하지 않을 시 배열 전체에 대한 소프트 맥스를 계산한다. 